In [246]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 2: Gift Shop ---</h2><p>You get inside and take the elevator to its only other stop: the gift shop. "Thank you for visiting the North Pole!" gleefully exclaims a nearby sign. You aren't sure who is even allowed to visit the North Pole, but you know you can access the lobby through here, and from there you can access the rest of the North Pole base.</p>
<p>As you make your way through the <span title="They even sell lunch boxes and blue tents!">surprisingly extensive</span> selection, one of the clerks recognizes you and asks for your help.</p>
<p>As it turns out, one of the younger Elves was playing on a gift shop computer and managed to add a whole bunch of invalid product IDs to their gift shop database! Surely, it would be no trouble for you to identify the invalid product IDs for them, right?</p>
<p>They've even checked most of the product ID ranges already; they only have a few product ID ranges (your puzzle input) that you'll need to check. For example:</p>
<pre><code>11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124</code></pre>
<p>(The ID ranges are wrapped here for legibility; in your input, they appear on a single long line.)</p>
<p>The ranges are separated by commas (<code>,</code>); each range gives its <em>first ID</em> and <em>last ID</em> separated by a dash (<code>-</code>).</p>
<p>Since the young Elf was just doing silly patterns, you can find the <em>invalid IDs</em> by looking for any ID which is made only of some sequence of digits repeated twice. So, <code>55</code> (<code>5</code> twice), <code>6464</code> (<code>64</code> twice), and <code>123123</code> (<code>123</code> twice) would all be invalid IDs.</p>
<p>None of the numbers have leading zeroes; <code>0101</code> isn't an ID at all. (<code>101</code> is a <em>valid</em> ID that you would ignore.)</p>
<p>Your job is to find all of the invalid IDs that appear in the given ranges. In the above example:</p>
<ul>
<li><code>11-22</code> has two invalid IDs, <code><em>11</em></code> and <code><em>22</em></code>.</li>
<li><code>95-115</code> has one invalid ID, <code><em>99</em></code>.</li>
<li><code>998-1012</code> has one invalid ID, <code><em>1010</em></code>.</li>
<li><code>1188511880-1188511890</code> has one invalid ID, <code><em>1188511885</em></code>.</li>
<li><code>222220-222224</code> has one invalid ID, <code><em>222222</em></code>.</li>
<li><code>1698522-1698528</code> contains no invalid IDs.</li>
<li><code>446443-446449</code> has one invalid ID, <code><em>446446</em></code>.</li>
<li><code>38593856-38593862</code> has one invalid ID, <code><em>38593859</em></code>.</li>
<li>The rest of the ranges contain no invalid IDs.</li>
</ul>
<p>Adding up all the invalid IDs in this example produces <code><em>1227775554</em></code>.</p>
<p><em>What do you get if you add up all of the invalid IDs?</em></p>
</article>


In [247]:
from itertools import batched
from math import log10


tests = [
    {
        "name": "Example",
        "s": """
            11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
            1698522-1698528,446443-446449,38593856-38593862,565653-565659,
            824824821-824824827,2121212118-2121212124
        """,
        "expected": 1227775554,
    },
]


def a_gen(n: int) -> int:
    return n * 10 ** (int(log10(n)) + 1) + n


def a_inverse(n: int, k: int = 2) -> int:
    l = int(log10(n)) + 1

    if l > 1 and l & 1 == 1:
        n = 10 ** (l - 1) - 1
        l = int(log10(n)) + 1

    l //= 2
    return n // (10**l + 1)


@test(tests=tests)
def part_I(s: str) -> int:
    ranges = batched((map(int, re.split(r"[,-]", re.sub(r"\s+", "", s)))), n=2)

    return sum(
        a
        for fr, to in ranges
        for n in range(a_inverse(fr), a_inverse(to) + 1)
        if fr <= (a := a_gen(n)) <= to
    )


Test Example passed, for part_I.
Success


In [248]:
with open("../input/day2.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 21898734247


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>21898734247</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The clerk quickly discovers that there are still invalid IDs in the ranges in your list. Maybe the young Elf was doing other silly patterns as well?</p>
<p>Now, an ID is invalid if it is made only of some sequence of digits repeated <em>at least</em> twice. So, <code>12341234</code> (<code>1234</code> two times), <code>123123123</code> (<code>123</code> three times), <code>1212121212</code> (<code>12</code> five times), and <code>1111111</code> (<code>1</code> seven times) are all invalid IDs.</p>
<p>From the same example as before:</p>
<ul>
<li><code>11-22</code> still has two invalid IDs, <code><em>11</em></code> and <code><em>22</em></code>.</li>
<li><code>95-115</code> now has two invalid IDs, <code><em>99</em></code> and <code><em>111</em></code>.</li>
<li><code>998-1012</code> now has two invalid IDs, <code><em>999</em></code> and <code><em>1010</em></code>.</li>
<li><code>1188511880-1188511890</code> still has one invalid ID, <code><em>1188511885</em></code>.</li>
<li><code>222220-222224</code> still has one invalid ID, <code><em>222222</em></code>.</li>
<li><code>1698522-1698528</code> still contains no invalid IDs.</li>
<li><code>446443-446449</code> still has one invalid ID, <code><em>446446</em></code>.</li>
<li><code>38593856-38593862</code> still has one invalid ID, <code><em>38593859</em></code>.</li>
<li><code>565653-565659</code> now has one invalid ID, <code><em>565656</em></code>.</li>
<li><code>824824821-824824827</code> now has one invalid ID, <code><em>824824824</em></code>.</li>
<li><code>2121212118-2121212124</code> now has one invalid ID, <code><em>2121212121</em></code>.</li>
</ul>
<p>Adding up all the invalid IDs in this example produces <code><em>4174379265</em></code>.</p>
<p><em>What do you get if you add up all of the invalid IDs using these new rules?</em></p>
</article>


## Part Two Analysis

For Part Two, we need to find all IDs that consist of a pattern repeated at least twice.

An ID is invalid if it can be expressed as: $N = S \cdot \sum_{i=0}^{k-1}(10^{L})^{i} = S \cdot \frac{10^{kL}-1}{10^{L}-1}$

Where:

- $S$ is the repeating sequence (as a number)
- $L$ is the length of the sequence
- $k \geq 2$ is the number of repetitions

For example:

- $11 = 1 \cdot \frac{10^{2}-1}{10^{1}-1} = 1 \cdot \frac{99}{9}$ (pattern "1" repeated 2 times)
- $123123 = 123 \cdot \frac{10^{6}-1}{10^{3}-1}$ (pattern "123" repeated 2 times)
- $1111111 = 1 \cdot \frac{10^{7}-1}{10^{1}-1}$ (pattern "1" repeated 7 times)

The algorithm:

1. For each range, iterate through all possible pattern lengths $L$
2. For each pattern length, find all valid repeating patterns in the range
3. Check if the constructed numbers fall within $[fr, to]$
4. Sum all invalid IDs found


In [251]:
from itertools import batched
from math import log10


tests = [
    {
        "name": "Example",
        "s": """
            11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
            1698522-1698528,446443-446449,38593856-38593862,565653-565659,
            824824821-824824827,2121212118-2121212124
        """,
        "expected": 4174379265,
    },
]


def a_gen(s: int, k: int) -> int:
    l = int(log10(s)) + 1
    return s * (10 ** (k * l) - 1) // (10**l - 1)


def a_inverse(n: int, k: int, pattern_length: int) -> int:
    # Find the minimum pattern value such that gen(pattern, k) >= n
    # pattern * ((10^(k*pattern_length) - 1) / (10^pattern_length - 1)) >= n
    # pattern >= n * (10^pattern_length - 1) / (10^(k*pattern_length) - 1)
    divisor = sum(10 ** (i * pattern_length) for i in range(k))
    return (n + divisor - 1) // divisor


@test(tests=tests)
def part_II(s: str) -> int:
    ranges = batched((map(int, re.split(r"[,-]", re.sub(r"\s+", "", s)))), n=2)

    count = 0
    for fr, to in ranges:
        l_to = int(log10(to)) + 1
        seen = set()

        for pattern_length in range(1, l_to // 2 + 1):
            for k in range(2, l_to // pattern_length + 1):
                k_fr = a_inverse(fr, k, pattern_length)
                k_to = a_inverse(to, k, pattern_length)

                n = k_fr
                while (
                    n <= k_to and n < 10**pattern_length
                ):  # Pattern must have correct length
                    a = a_gen(n, k)
                    if fr <= a <= to and a not in seen:
                        seen.add(a)
                        count += a

                    n += 1

    return count


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [252]:
print(f"Part II: {part_II(puzzle)}")

Part II: 28915664389


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>28915664389</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
